In [11]:
import pandas as pd

demographics = pd.read_csv("us_census/demographics.csv")
demographics.head()

,zip,lat,lng,city,state_id,state_name,population,density,county_name,po_box,dist_highway,dist2_large_airport,dist2_medium_airport,dist_to_shore,number_of_business,adjusted_gross_income,total_income_amount,number_of_returns
0,1001,42.06262,-72.62521,Agawam,MA,Massachusetts,16088,550.1,Hampden,0,1.387035,106.145765,12.946212,93.049251,438.0,598807,6.047690e+05,9320
1,1002,42.37633,-72.46462,Amherst,MA,Massachusetts,27323,198.1,Hampshire,0,14.438177,112.264368,21.080079,133.370144,571.0,989558,1.005796e+06,9880
2,1005,42.42117,-72.10655,Barre,MA,Massachusetts,4947,44.2,Worcester,0,16.788339,90.664964,25.547718,97.639881,97.0,164207,1.660540e+05,2490
3,1007,42.28163,-72.40009,Belchertown,MA,Massachusetts,15304,107.7,Hampshire,0,13.663839,101.552921,14.762395,114.406034,217.0,647074,6.547390e+05,7970
4,1008,42.18234,-72.95819,Blandford,MA,Massachusetts,1171,7.4,Hampden,0,2.593655,136.548797,20.177950,107.466779,18.0,47826,4.824100e+04,660


In [12]:
demographics = demographics[demographics['state_id'] == "PA"]
demographics = demographics[['zip', 'dist_highway', 'adjusted_gross_income']]
demographics = demographics.rename(columns={"zip":"postal_code"})
demographics.head()

,postal_code,dist_highway,adjusted_gross_income
3725,15001,2.038176,1019897
3726,15003,6.920580,276826
3727,15005,6.838959,457858
3728,15007,4.827754,13687
3729,15009,0.097311,623705


In [13]:
unemployment = pd.read_csv("us_census/unemployment.csv")
unemployment.head()

,FIPS_Code,State,Area_Name,Attribute,Value
0,0,US,United States,Civilian_labor_force_2000,142601576.0
1,0,US,United States,Employed_2000,136904853.0
2,0,US,United States,Unemployed_2000,5696723.0
3,0,US,United States,Unemployment_rate_2000,4.0
4,0,US,United States,Civilian_labor_force_2001,143786537.0


In [14]:
unemployment = unemployment[unemployment['State'] == "PA"]
# Get only 2023 data
unemployment = unemployment[unemployment['Attribute'].str.contains(
    '2023.*') == True].reset_index()
unemployment = unemployment.pivot(
    index=['FIPS_Code', 'State'], columns='Attribute', values='Value')
unemployment = unemployment.reset_index()
unemployment.columns.name = None
unemployment.head()

,FIPS_Code,State,Civilian_labor_force_2023,Employed_2023,Metro_2023,Rural_Urban_Continuum_Code_2023,Unemployed_2023,Unemployment_rate_2023
0,42000,PA,6518343.0,6295653.0,NaN,NaN,222690.0,3.4
1,42001,PA,55049.0,53619.0,1.0,3.0,1430.0,2.6
2,42003,PA,632239.0,611346.0,1.0,1.0,20893.0,3.3
3,42005,PA,31079.0,29858.0,1.0,1.0,1221.0,3.9
4,42007,PA,81676.0,78350.0,1.0,1.0,3326.0,4.1


In [15]:
zipfip = pd.read_csv("us_census/zip_fip.csv")
zipfip.head()

,ZIP,COUNTYNAME,STATE,STCOUNTYFP,CLASSFP
0,36003,Autauga County,AL,1001,H1
1,36006,Autauga County,AL,1001,H1
2,36067,Autauga County,AL,1001,H1
3,36066,Autauga County,AL,1001,H1
4,36703,Autauga County,AL,1001,H1


In [16]:
zipfip = zipfip[zipfip["STATE"] == "PA"]
zipfip.head()

,ZIP,COUNTYNAME,STATE,STCOUNTYFP,CLASSFP
38479,17307,Adams County,PA,42001,H1
38480,17340,Adams County,PA,42001,H1
38481,17310,Adams County,PA,42001,H1
38482,17350,Adams County,PA,42001,H1
38483,17344,Adams County,PA,42001,H1


In [17]:
unemployment = zipfip.merge(unemployment, left_on='STCOUNTYFP',
             right_on='FIPS_Code', how='left')
unemployment = unemployment[['ZIP', 'Rural_Urban_Continuum_Code_2023', 'Unemployment_rate_2023']]
unemployment = unemployment.rename(columns={
                                   "ZIP": "postal_code", 
                                   "Rural_Urban_Continuum_Code_2023": "rural_urban_continuum_code_2023",
                                   "Unemployment_rate_2023": "unemployment_rate_2023"})
unemployment.head()

,postal_code,rural_urban_continuum_code_2023,unemployment_rate_2023
0,17307,3.0,2.6
1,17340,3.0,2.6
2,17310,3.0,2.6
3,17350,3.0,2.6
4,17344,3.0,2.6


In [18]:
df = demographics.merge(unemployment, left_on='postal_code', right_on='postal_code', how='left')
df.head()

,postal_code,dist_highway,adjusted_gross_income,rural_urban_continuum_code_2023,unemployment_rate_2023
0,15001,2.038176,1019897,1.0,3.3
1,15001,2.038176,1019897,1.0,4.1
2,15003,6.920580,276826,1.0,3.3
3,15003,6.920580,276826,1.0,4.1
4,15005,6.838959,457858,1.0,3.3


In [19]:
df.to_csv('us_census/demographics_merged.csv', index=False)

In [22]:
df.info()
df.describe().round(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2301 entries, 0 to 2300
Data columns (total 5 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   postal_code                      2301 non-null   int64  
 1   dist_highway                     2301 non-null   float64
 2   adjusted_gross_income            2301 non-null   int64  
 3   rural_urban_continuum_code_2023  2299 non-null   float64
 4   unemployment_rate_2023           2299 non-null   float64
dtypes: float64(3), int64(2)
memory usage: 90.0 KB


,postal_code,dist_highway,adjusted_gross_income,rural_urban_continuum_code_2023,unemployment_rate_2023
count,2301.00,2301.00,2301.00,2299.00,2299.00
mean,17118.15,9.14,333684.18,3.17,3.59
std,1342.19,9.11,522495.70,2.28,0.55
min,15001.00,0.00,2306.00,1.00,2.60
25%,15925.00,2.47,43791.00,1.00,3.20
50%,17026.00,6.33,117046.00,2.00,3.50
75%,18239.00,13.09,385441.00,4.00,4.00
max,19611.00,69.31,3836907.00,9.00,5.20
